### Zadanie 1 - Paradoks Simpsona##

Paradoks Simpsona mówi o tym, że badając pewne zjawisko możemy otrzymywać inne wyniki w przypadku rozpatrywania badanych grup odzielnie a inne, często odwrotne, jeśli dokonamy połączenia badanych grup.

#### Przykład:  Psy czy koty? ####

Chcemy sprawdzić które ze zwierzaków są bardziej popularne, więc przepytujemy znajomych. Wśród nich mamy zarówno kobiety jak i mężczyzn, więc dzielimy wyniki naszego wywiadu na dwie grupy. Okazuje się, że wśród naszych kolegów psy cieszą się dużo większą popularnością - aż 80%, podczas gdy tylko 40% koleżanek preferuje psy. Tak więc koty lubi zaledwie 20% naszych przyjaciół oraz 60% przyjaciółek. Wydawać by się mogło, że większość naszych przyjaciół wolałaby mieć psa. Jednak po podsumowaniu wyników okazuje się, że podział na fanów kotów i psów wśród znajomych jest prawie 50:50.

In [31]:
import pandas as pd
import numpy as np
col = ['M', 'K', 'Razem']
zw = ['Psy', 'Koty']
df = pd.DataFrame(np.array(['80%', '40%', '48%' ,'20%', '60%', '52%']).reshape((2,3)), columns=col, index = zw)
df

,M,K,Razem
Psy,80%,40%,48%
Koty,20%,60%,52%


Skąd taka różnica? Wynika z różnej liczebności obu grup: kolegów mamy łącznie 5 a koleżanek aż 20.

In [30]:
df2 = pd.DataFrame(np.array([4, 8, 12, 1, 12, 13]).reshape((2,3)), columns=col, index=zw)
df2

,M,K,Razem
Psy,4,8,12
Koty,1,12,13


#### Wpływ na metody pomiaru kampanii marketingowych ####

Wystarczy, że w powyższym przykładzie zastąpimy Psy i Koty dwiema kampaniami marketingowymi przeprowadzanymi na różnych grupach docelowych - chociażby na mężczyznach i kobietach. Jeśli uznamy, że na pierwszą kampanię marketingową odpowiedziało 80% mężczyzn i 40% kobiet a na drugą 20% mężczyzn i 60% kobiet, bez podawania liczebności każdej z płci możemy otrzymać błędne oszacowanie skuteczności kampanii - które w rzeczywistości są prawie tak samo skuteczne.

---

### Zadanie 2 - Raport skuteczności kampanii marketingowej ###

Aby ocenić skuteczność kampanii marketingowej musimy sprawdzić, czy średnia wartość zakupów w serwisie Allegro w okresie od 1 lutego do 31 marca była istotnie większa wśród osób do których skierowano kampanię w porównaniu do tych, do których nie wysłano materiałów zachęcających do dokonania zakupów.

1\. Stworzenie zbioru składającego się z klientów, którzy zrobili zakupy na Allegro w ciągu tych dwóch miesięcy wraz z sumaryczną kwotą wydatków w tym okresie:
Wybranie tych dla których ID_Sklepu = alg oraz Data_Trx Between('2013-02-01','2013-03-31')

In [38]:
x1 = pd.ExcelFile('zadanie.xlsx')
x1.sheet_names

[u'Tabela_transakcje', u'Tabela_historia_wysylki']

In [42]:
trans = x1.parse('Tabela_transakcje')
hist = x1.parse('Tabela_historia_wysylki')
trans.head()

,ID_KLIENTA,ID_SKLEPU,DATA_TRX,KWOTA,PUNKTY
0,486,bp,2013-05-11,346.30,519
1,874,bp,2013-03-21,174.96,87
2,874,bp,2013-04-10,137.92,68
3,1844,real,2013-01-04,95.80,47
4,2426,real,2013-02-20,158.82,79


In [54]:
allegro = trans.loc[trans['ID_SKLEPU']=='alg']
allegro = allegro.loc[allegro['DATA_TRX'].isin(pd.date_range('2013-02-01', '2013-03-31'))]
allegro[0:5]

,ID_KLIENTA,ID_SKLEPU,DATA_TRX,KWOTA,PUNKTY
26,5142,alg,2013-02-24,26.04,13
28,5142,alg,2013-03-13,26.04,13
30,5142,alg,2013-03-20,12.17,6
31,5142,alg,2013-03-25,22.78,33
32,5142,alg,2013-03-26,19.15,9


In [83]:
all_sum = pd.DataFrame(allegro.groupby('ID_KLIENTA')['KWOTA'].sum())
all_sum[0:1]

,KWOTA
ID_KLIENTA,
5142,106.18


2\. Podział na grupę kontrolną oraz grupę odbiorców materiałów promocyjnych.

In [71]:
hist.head()

,ID_KLIENTA,STATUS_WYSYLKI
0,5045,GRUPA_KONTROLNA
1,5627,WYSLANO
2,5918,WYSLANO
3,8828,WYSLANO
4,13678,WYSLANO


In [91]:
gr_kontr = hist.loc[hist['STATUS_WYSYLKI']=='GRUPA_KONTROLNA']
gr_odb = hist.loc[hist['STATUS_WYSYLKI']=='WYSLANO']
gr_kontr_sum = pd.merge(gr_kontr, all_sum, left_on='ID_KLIENTA',right_index=True ,how='outer')
_ = gr_kontr_sum['KWOTA'].fillna(0, inplace=True)
gr_odb_sum = pd.merge(gr_odb, all_sum, left_on='ID_KLIENTA',right_index=True ,how='outer')
gr_odb_sum['ID_KLIENTA'].count()


416

3\. Dla każdego z dwóch nowych zbiorów policzenie dla każdego klienta kwoty którą wydał w ciagu tych dwóch miesięcy za pomocą połączenia ze zbiorem z transakcjami.

4\. Policzenie średnich dla obu grup - kontrolnej i odbiorców. Zbadanie hipotezy o równości średnich.